In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.singleton_classifier import SingletonClassifierModelBuilder
from functools import reduce

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
data = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

data.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1,[1],0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 12, 13, 14]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]",1,0,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 12, 13, 14, 15, 16, 17]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[0.0, 1.0]"
4,5,"[12, 13]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[14, 15, 16, 17, 18, 19, 4, 5, 20, 21]","[0.0, 1.0]"


In [4]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
label = np.vstack(data.is_singleton)

# Build Model

## Words

In [5]:
words_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [6]:
words_model = words_model_builder.create_model()

W0314 11:20:07.272945 139668807812928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:111: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0314 11:20:08.513025 139668807812928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:135: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0314 11:20:08.570456 139668807812928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1253: calling VarianceScaling.__init__ (from tensorflow.python.op

In [7]:
words_model.fit([data_text], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 2s 411us/sample - loss: 0.3195 - acc: 0.8555
Epoch 2/20
5032/5032==============================] - 1s 211us/sample - loss: 0.2180 - acc: 0.9026
Epoch 3/20
5032/5032==============================] - 1s 213us/sample - loss: 0.1724 - acc: 0.9175
Epoch 4/20
5032/5032==============================] - 1s 213us/sample - loss: 0.1424 - acc: 0.9332
Epoch 5/20
5032/5032==============================] - 1s 216us/sample - loss: 0.1246 - acc: 0.9396
Epoch 6/20
5032/5032==============================] - 1s 211us/sample - loss: 0.1226 - acc: 0.9438
Epoch 7/20
5032/5032==============================] - 1s 211us/sample - loss: 0.1120 - acc: 0.9448
Epoch 8/20
5032/5032==============================] - 1s 213us/sample - loss: 0.1048 - acc: 0.9461
Epoch 9/20
5032/5032==============================] - 1s 209us/sample - loss: 0.1052 - acc: 0.9477
Epoch 10/20
5032/5032==============================] - 1s 212us/sample - loss: 0.0963 - acc: 0.9483
Epoch 11/

In [8]:
words_model.save('models/singleton_classifiers/words.model')

## Context

In [9]:
context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [10]:
context_model = context_model_builder.create_model()

In [11]:
context_model.fit([data_previous_words, data_next_words], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 2s 336us/sample - loss: 0.5226 - acc: 0.8086
Epoch 2/20
5032/5032==============================] - 1s 292us/sample - loss: 0.4339 - acc: 0.8382
Epoch 3/20
5032/5032==============================] - 1s 293us/sample - loss: 0.3717 - acc: 0.8480
Epoch 4/20
5032/5032==============================] - 1s 291us/sample - loss: 0.3226 - acc: 0.8611
Epoch 5/20
5032/5032==============================] - 1s 293us/sample - loss: 0.2828 - acc: 0.8702
Epoch 6/20
5032/5032==============================] - 1s 290us/sample - loss: 0.2275 - acc: 0.8899
Epoch 7/20
5032/5032==============================] - 1s 292us/sample - loss: 0.2041 - acc: 0.9060
Epoch 8/20
5032/5032==============================] - 1s 288us/sample - loss: 0.1875 - acc: 0.9112
Epoch 9/20
5032/5032==============================] - 1s 292us/sample - loss: 0.1391 - acc: 0.9400
Epoch 10/20
5032/5032==============================] - 1s 294us/sample - loss: 0.1354 - acc: 0.9469
Epoch 11/

In [12]:
context_model.save('models/singleton_classifiers/context.model')

## Syntactic

In [13]:
syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    syntactic_features_num=data_syntactic.shape[1]
)

In [14]:
syntactic_model = syntactic_model_builder.create_model()

In [15]:
syntactic_model.fit([data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 1s 182us/sample - loss: 0.4004 - acc: 0.8349
Epoch 2/20
5032/5032==============================] - 1s 153us/sample - loss: 0.2681 - acc: 0.8454
Epoch 3/20
5032/5032==============================] - 1s 153us/sample - loss: 0.2550 - acc: 0.8569
Epoch 4/20
5032/5032==============================] - 1s 150us/sample - loss: 0.2497 - acc: 0.8529
Epoch 5/20
5032/5032==============================] - 1s 156us/sample - loss: 0.2519 - acc: 0.8527
Epoch 6/20
5032/5032==============================] - 1s 157us/sample - loss: 0.2493 - acc: 0.8569
Epoch 7/20
5032/5032==============================] - 1s 152us/sample - loss: 0.2505 - acc: 0.8575
Epoch 8/20
5032/5032==============================] - 1s 154us/sample - loss: 0.2478 - acc: 0.8623
Epoch 9/20
5032/5032==============================] - 1s 155us/sample - loss: 0.2479 - acc: 0.8619
Epoch 10/20
5032/5032==============================] - 1s 153us/sample - loss: 0.2470 - acc: 0.8625
Epoch 11/

In [16]:
syntactic_model.save('models/singleton_classifiers/syntactic.model')

## Words + Context

In [17]:
words_context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [18]:
words_context_model = words_context_model_builder.create_model()

In [19]:
words_context_model.fit([data_text, data_previous_words, data_next_words], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 2s 430us/sample - loss: 0.3177 - acc: 0.8547
Epoch 2/20
5032/5032==============================] - 2s 359us/sample - loss: 0.1896 - acc: 0.9141
Epoch 3/20
5032/5032==============================] - 2s 363us/sample - loss: 0.1257 - acc: 0.9473
Epoch 4/20
5032/5032==============================] - 2s 364us/sample - loss: 0.0957 - acc: 0.9620
Epoch 5/20
5032/5032==============================] - 2s 357us/sample - loss: 0.0736 - acc: 0.9728
Epoch 6/20
5032/5032==============================] - 2s 357us/sample - loss: 0.0710 - acc: 0.9734
Epoch 7/20
5032/5032==============================] - 2s 359us/sample - loss: 0.0604 - acc: 0.9767
Epoch 8/20
5032/5032==============================] - 2s 358us/sample - loss: 0.0578 - acc: 0.9781
Epoch 9/20
5032/5032==============================] - 2s 359us/sample - loss: 0.0463 - acc: 0.9821
Epoch 10/20
5032/5032==============================] - 2s 357us/sample - loss: 0.0359 - acc: 0.9841
Epoch 11/

In [20]:
words_context_model.save('models/singleton_classifiers/words_context.model')

## Words + Syntactic

In [21]:
words_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [22]:
words_syntactic_model = words_syntactic_model_builder.create_model()

In [23]:
words_syntactic_model.fit([data_text, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 1s 292us/sample - loss: 0.3861 - acc: 0.8031
Epoch 2/20
5032/5032==============================] - 1s 242us/sample - loss: 0.1883 - acc: 0.9042
Epoch 3/20
5032/5032==============================] - 1s 245us/sample - loss: 0.1517 - acc: 0.9217
Epoch 4/20
5032/5032==============================] - 1s 249us/sample - loss: 0.1266 - acc: 0.9338
Epoch 5/20
5032/5032==============================] - 1s 247us/sample - loss: 0.1182 - acc: 0.9404
Epoch 6/20
5032/5032==============================] - 1s 246us/sample - loss: 0.1159 - acc: 0.9366
Epoch 7/20
5032/5032==============================] - 1s 247us/sample - loss: 0.1079 - acc: 0.9422
Epoch 8/20
5032/5032==============================] - 1s 246us/sample - loss: 0.1075 - acc: 0.9374
Epoch 9/20
5032/5032==============================] - 1s 248us/sample - loss: 0.0994 - acc: 0.9485
Epoch 10/20
5032/5032==============================] - 1s 247us/sample - loss: 0.0989 - acc: 0.9446
Epoch 11/

In [24]:
words_syntactic_model.save('models/singleton_classifiers/words_syntactic.model')

## Context + Syntactic

In [25]:
context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [26]:
context_syntactic_model = context_syntactic_model_builder.create_model()

In [27]:
context_syntactic_model.fit([data_previous_words, data_next_words, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 2s 382us/sample - loss: 0.4486 - acc: 0.8106
Epoch 2/20
5032/5032==============================] - 2s 312us/sample - loss: 0.2524 - acc: 0.8786
Epoch 3/20
5032/5032==============================] - 2s 318us/sample - loss: 0.1965 - acc: 0.9141
Epoch 4/20
5032/5032==============================] - 2s 320us/sample - loss: 0.1541 - acc: 0.9324
Epoch 5/20
5032/5032==============================] - 2s 321us/sample - loss: 0.1213 - acc: 0.9473
Epoch 6/20
5032/5032==============================] - 2s 316us/sample - loss: 0.1094 - acc: 0.9549
Epoch 7/20
5032/5032==============================] - 2s 313us/sample - loss: 0.0883 - acc: 0.9634
Epoch 8/20
5032/5032==============================] - 2s 318us/sample - loss: 0.0815 - acc: 0.9654
Epoch 9/20
5032/5032==============================] - 2s 320us/sample - loss: 0.0687 - acc: 0.9688
Epoch 10/20
5032/5032==============================] - 2s 318us/sample - loss: 0.0593 - acc: 0.9767
Epoch 11/

In [28]:
context_syntactic_model.save('models/singleton_classifiers/context_syntactic.model')

## Words + Context + Syntactic

In [29]:
words_context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [30]:
words_context_syntactic_model = words_context_syntactic_model_builder.create_model()

In [31]:
words_context_syntactic_model.fit([data_text, data_previous_words, data_next_words, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032==============================] - 2s 467us/sample - loss: 0.3937 - acc: 0.8269
Epoch 2/20
5032/5032==============================] - 2s 387us/sample - loss: 0.1950 - acc: 0.9070
Epoch 3/20
5032/5032==============================] - 2s 392us/sample - loss: 0.1435 - acc: 0.9360
Epoch 4/20
5032/5032==============================] - 2s 385us/sample - loss: 0.0998 - acc: 0.9591
Epoch 5/20
5032/5032==============================] - 2s 386us/sample - loss: 0.0939 - acc: 0.9658
Epoch 6/20
5032/5032==============================] - 2s 389us/sample - loss: 0.0746 - acc: 0.9690
Epoch 7/20
5032/5032==============================] - 2s 390us/sample - loss: 0.0612 - acc: 0.9758
Epoch 8/20
5032/5032==============================] - 2s 388us/sample - loss: 0.0655 - acc: 0.9748
Epoch 9/20
5032/5032==============================] - 2s 386us/sample - loss: 0.0577 - acc: 0.9766
Epoch 10/20
5032/5032==============================] - 2s 386us/sample - loss: 0.0573 - acc: 0.9785
Epoch 11/

In [32]:
words_context_syntactic_model.save('models/singleton_classifiers/words_context_syntactic.model')